In [10]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [12]:
import pickle

road_df=pd.read_csv('wokmeans_exp15.csv').iloc[:, 1:]
road_df=StandardScaler().fit_transform(road_df)
pca = PCA(n_components=4)
model=pca.fit(road_df)
    
print(model.explained_variance_ratio_)
#print(model.components_)
#road_Df = pd.DataFrame(data = model)
#print(road_Df)
pickle.dump(model, open('pca_model.pkl', 'wb')) 

[0.11019543 0.10123451 0.07515242 0.07203776]


In [16]:
main_folder=r"D:\TYETA\CV\Data_Set"

for folder in os.listdir(main_folder):
    model = pickle.load(open('pca_model.pkl', "rb"))
    path=os.path.join(main_folder,folder)
    for filename in os.listdir(path):
        p=os.path.join(path,filename)
        a=cv2.imread(p)
        resize=(300,300)
        img=cv2.resize(a,resize)#resize image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        orb = cv2.ORB_create(150)#initialise  detector
        keypoints, descriptors = orb.detectAndCompute(gray, None)
        out=pd.DataFrame(descriptors)
        
        output=pd.DataFrame(model.fit_transform(out))
        hist=np.histogram(output.values,bins=[0,1,2,3,4,5])
        histout=pd.DataFrame(hist[0])
        
    csv_data=histout.transpose().to_csv(str(folder)+'_pca.csv', mode='a', header=False)
        
    print('PCA_prediction of ' + str(folder) +' images Done!')


PCA_prediction of Cement Road images Done!
PCA_prediction of Kaccha Road images Done!
PCA_prediction of Paver Road images Done!
PCA_prediction of Tarcoal Road images Done!


In [17]:
from csv import writer
from csv import reader

main_folder=r"D:\TYETA\CV\Data_Set"
i=1
for folder in os.listdir(main_folder):
    with open(str(folder)+'_pca.csv', 'r') as read_obj, \
        open('FinalORB_pca.csv', 'a+', newline='') as write_obj:
        csv_reader = reader(read_obj)     # Create a csv.reader object from the input file object
        csv_writer = writer(write_obj)    # Create a csv.writer object from the output file object
        # Read each row of the input csv file as list
        for row in csv_reader:
            row.append(i)            # Append the class in the row / list
            csv_writer.writerow(row)      # Add the updated row / list to the output file
    i+=1

In [19]:
dataset = pd.read_csv('FinalORB_pca.csv')
X = dataset.iloc[:, 1:6].values
y = dataset.iloc[:, 6].values

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

In [21]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [34]:
from sklearn import svm
clf = svm.SVC(kernel='poly') # Linear Kernel

clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [35]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.2975206611570248
